In [1]:
import os
import torch
import torch.nn as nn

In [2]:
# Расположение файла для кодировки слов
path_vocab = 'C:\\Users\\drug1\\Anaconda\\Keras\\films\\imdb.vocab'

In [3]:
##### функция преобразования текста в числа

def transform_text(text1, path_vocab=path_vocab, len_text=280):
    fl = False 
    with open(path_vocab, "r") as f1:
        vocab = f1.read().splitlines()
    vocab = vocab[:39998]  
    mass = [] 
    str1 =''    
    for ch in text1:
        if len(mass)>len_text-1: # если слов больше чем нужно, выходим
            data_tensor = torch.tensor(mass)
            return data_tensor
        if ch !=' ':
            str1 = str1 + ch
        if ch ==' ':
            if str1 !='':                      
                fl = False                                                            
                for i in range(len(vocab)):
                    if str1.lower() == vocab[i].lower():
                        fl = True
                        mass.append(i+2)
                        str1 =''
                        break                                                 
                if fl == False: #  если слово не найдено, заменяем нулями
                    mass.append(0)
                    str1 =''        
    for i in range(len(vocab)): # Проверяем последнее слово
        if str1.lower() == vocab[i].lower():
            fl = True
            mass.append(i+2)
            str1 =''
            break                                                 
    if fl == False: #  если слово не найдено, заменяем нулями
        mass.append(0)
        str1 =''

    if len(mass)<len_text:  #  если слов меньше чем нужно, добавляем нулями.
        while len(mass)<len_text:
            mass.insert(0, 0)
            str1 =''       
    data_tensor = torch.tensor(mass)    
    return data_tensor

In [4]:

# Определение класса модели
class SentimentClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SentimentClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm1 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)       
        self.dropout = nn.Dropout(p=0.2)  # Добавление слоя Dropout
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm1(embedded)
        output, _ = self.lstm2(output)
        #output = self.dropout(output)  # Применение Dropout к выходу LSTM
        #output = self.fc(output[:, -1, :])  # Используется только последний выход LSTM
        
        output = output[-1, :]  # Получение последнего выхода LSTM
        output = self.fc(output)  # Применение полносвязного слоя
        return output

In [12]:
input_size = 40000 
hidden_size = 256
num_classes = 8

model = SentimentClassifier(input_size, hidden_size, num_classes).to('cpu')

In [13]:

model.load_state_dict(torch.load('model_3.pth'))


model.eval()

SentimentClassifier(
  (embedding): Embedding(40000, 256)
  (lstm1): LSTM(256, 256, batch_first=True)
  (lstm2): LSTM(256, 256, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=8, bias=True)
)

In [14]:
text = 'This is easily the most underrated film inn the Brooks cannon. Sure, its flawed. It does not give a realistic view of homelessness (unlike, say, how Citizen Kane gave a realistic view of lounge singers, or Titanic gave a realistic view of Italians YOU IDIOTS). Many of the jokes fall flat. But still, this film is very lovable in a way many comedies are not, and to pull that off in a story about some of the most traditionally reviled members of society is truly impressive. Its not The Fisher King, but its not crap, either. My only complaint is that Brooks should have cast someone else in the lead (I love Mel as a Director and Writer, not so much as a lead).'

In [15]:
text

'This is easily the most underrated film inn the Brooks cannon. Sure, its flawed. It does not give a realistic view of homelessness (unlike, say, how Citizen Kane gave a realistic view of lounge singers, or Titanic gave a realistic view of Italians YOU IDIOTS). Many of the jokes fall flat. But still, this film is very lovable in a way many comedies are not, and to pull that off in a story about some of the most traditionally reviled members of society is truly impressive. Its not The Fisher King, but its not crap, either. My only complaint is that Brooks should have cast someone else in the lead (I love Mel as a Director and Writer, not so much as a lead).'

In [16]:

text2 = transform_text(text)

In [17]:
type(text2)

torch.Tensor

In [18]:
output = model(text2)

In [19]:
output

tensor([-4.4222, -4.8097, -0.0797, -0.7776, 13.5095, -1.8159, -4.3296, -1.9749],
       grad_fn=<AddBackward0>)

In [20]:
import torch.nn.functional as F

In [21]:
probs = F.softmax(output, dim=0)
predicted_class = torch.argmax(probs, dim=0)

print(predicted_class.item())

4
